# Mehr zu pandas für DataFrames

## Einleitung

Häufig haben wir es in den Digital Humanities mit Daten in einem tabellarischen Format, bspw. in Form einer CSV-Datei, zu tun. Oft sind das Metadaten oder auch Ergebnisse von Berechnungen. 

Solche Tabellen lassen sich in Python sehr gut mit pandas als DataFrame öffnen, analysieren und visualisieren. Das probieren wir hier im Sinne einer vertiefenden Einführung in DataFrames mit einem realen Beispieldatensatz vom 'Index of DH Conferences' aus. 

## Importe

In [ ]:
import pandas as pd
from os.path import join

## Eine CSV-Datei einlesen

- Öffnen der Datei mit `with open`
- Einlesen als DataFrame mit `read_csv` (Parameter: `sep` für Separator und `index_col` für die Spalte, die als Index verwendet werden soll)
- Weitere Infos zu den Parametern: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
csvfile = join("..", "data", "idhc_conferences.csv")

def read_csvfile(csvfile): 
    with open(csvfile, "r", encoding="utf8") as infile: 
        data = pd.read_csv(infile, sep=",", index_col=0)
    return data

data = read_csvfile(csvfile)

## Einen Eindruck von Umfang und Inhalt bekommen

- Größe der Tabelle mit `.shape`
- Liste der Spaltentitel mit `.columns`
- Index mit `.index`
- Anfang der Tabelle mit `.head()`
- Datentypen der Spalten mit `.dtypes`


In [ ]:
print("shape", data.shape)
print("columns", list(data.columns))
print("index", list(data.index[0:50]))
#print(data.head())
#print(data.dtypes)

## Teile der Tabelle auswählen

- Slicing nach Index-Positionen mit `iloc`
- Slicing mit Labels mit `loc`


In [ ]:
def select_from_data(data): 
    # mit iloc
    selected = data.iloc[0:3,0:3] # [Zeilen,Spalten]
    selected = data.iloc[15:20,5:8] 
    selected = data.iloc[100:105,:] 
    selected = data.iloc[:,3:5] 
    selected = data.loc[:,"city"]
    selected = data.loc[112, "city"] # Index=int!
    selected = data.loc[[48,112], "city"] # Liste!
    selected = data.loc[[1,2,3,4], ["start_date", "end_date"]] # Liste!
    return selected

selected = select_from_data(data)
print(type(selected))
print(selected.shape)
print(selected)

## Die Tabelle nach bestimmten Kriterien filtern

- Bestimmte Zeilen oder Spalten löschen mit `.drop()`
- Zeilen löschen, wenn eine Zelle einen bestimmten Wert hat

In [ ]:
# Zeilen, die in der Spalte "city" den Wert "London" haben
filtered = data[data["city"] == "London"]

# Zwei Kriterien: Stadt (string) und Jahr (int)
#filtered = data[(data["city"] == "Berlin") & (data["year"] >= 1995)]

print(filtered.shape) # Zeilen,Spalten
print(list(filtered.loc[:,"year"])) # Welche Jahre, als Liste
#print(filtered.head())


## Berechnungen auf der Tabelle ausführen

Dafür ist es oft nützlich, numpy zu verwenden. 

- Mittelwert, Median, Standardabweichung einer Zeile oder Spalte berechnen
- Andere Berechnung auf der Grundlage von zwei Zellen pro Zeile
- Ergebnis als neue Spalte zum DataFrame hinzufügen
- Hier am Beispiel der Konferenzdaten, daher siehe auch: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html

In [ ]:
import numpy as np

# Median / Mittelwert der Konferenzjahre
#result = np.median(data.loc[:,"year"])
result = np.mean(data.loc[:,"year"])
#print(result)

# Wie viele Konferenzen gab es denn pro Jahr? Mit Sortierung
yearcounts = data["year"].value_counts()
#print(type(yearcounts))
yearcounts.sort_values(inplace=True, ascending=False)
yearcounts.sort_index(inplace=True, ascending=False)
#print(yearcounts)

# Mittlere Dauer der Konferenzen in Tagen
filtered = data.loc[:,["start_date", "end_date"]]
filtered.dropna(inplace=True) # Unvollständige Einträge löschen
#print(filtered.head())
duration = pd.to_datetime(data["end_date"], errors="ignore") - pd.to_datetime(data["start_date"], errors="ignore")
print(duration)
print(sorted(list(duration.dt.days)))

# Dauer in Tagen der Tabelle hinzufügen
filtered["duration"] = duration.dt.days
#print(filtered.head())




## Einen DataFrame abspeichern

Den eben erstellen Dataframe mit den Konferenzdauern wollen wir jetzt abspeichern. 

- eine neue Datei anlegen mit `with open()`
- Einen DataFrame als CSV herauschreiben mit `.to_csv()`

In [ ]:
def save_dataframe(filtered): 
    with open("meinetabelle.csv", "w", encoding="utf8") as outfile: 
        filtered.to_csv(outfile, sep=";")

save_dataframe(filtered)

## Weitere Themen

- Informationen aus einem DataFrame visualisieren (direkt in pandas mit `.plot()` oder beispielsweise mit der Library seaborn). 
- Einen DataFrame nach einem bestimmten Kriterium in mehrere Teile aufteilen mit `.groupby`. 
- Mehrere DataFrames miteinander verbinden mit `merge`, `concat` und/oder `join`. 
- Einen DataFrame aus mehreren Listen (mit `dict(zip[])`), aus einem Dictionary (mit `from_dict`) oder aus mehreren `Series` erstellen. 